<a href="https://colab.research.google.com/github/jasonjstewart/college-football/blob/Jason/EDA_College_Football_4th_Downs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
df=pd.read_csv('https://jasonstewart.s3.amazonaws.com/4th-down-data.csv')
#THIS IS AWESOME WOOHOO
df.drop(['yard_line.1','yards_to_goal.1','play_text','offense.1','offense_conference.1','defense.1','defense_conference.1','id.2','start_date','home_id','home_team','home_points','away_team','away_conference','away_points','home_conference'],axis=1, inplace=True)
df.describe()
df.head(3)

,offense,id,offense_conference,defense,defense_conference,home,away,offense_score,defense_score,drive_id,period,clock,yard_line,yards_to_goal,down,distance,yards_gained,play_type,ppa,id.1,game_id,scoring,start_period,start_yardline,start_yards_to_goal,start_time,end_period,end_yardline,end_yards_to_goal,end_time,elapsed,plays,yards,drive_result,season,week,season_type,neutral_site,conference_game,attendance,home_post_win_prob,away_id,away_post_win_prob,excitement_index,success_or_failure
0,Georgia,4.006040e+17,SEC,Auburn,SEC,Auburn ...,Georgia,0,7,4006039314,1,89,36,36,4,4,5,Pass Reception,2.130077,4006039314,400603931,False,1,66,66,"{'minutes': 4, 'seconds': 32}",2,2,2,"{'minutes': 12, 'seconds': 39}","{'minutes': 6, 'seconds': 53}",12,64,DOWNS,2015,11,regular,False,True,87451,0.6809,61,0.3191,3.706368,0
1,Vanderbilt,4.010120e+17,SEC,Tennessee State,None,Vanderbilt ...,Tennessee State,0,0,4010122992,1,696,68,32,4,2,-10,Pass Interception Return,-2.672218,4010122992,401012299,False,1,49,51,"{'minutes': 11, 'seconds': 36}",1,68,32,"{'minutes': 8, 'seconds': 17}","{'minutes': 3, 'seconds': 19}",6,19,INT,2018,5,regular,False,False,0,0.6333,2634,0.3667,6.214617,0
2,South Carolina,4.006040e+17,SEC,LSU,SEC,LSU ...,South Carolina,3,7,4006039033,1,315,56,56,4,2,0,Pass Incompletion,-0.589364,4006039033,400603903,False,1,75,75,"{'minutes': 5, 'seconds': 15}",1,56,56,"{'minutes': 2, 'seconds': 37}","{'minutes': 2, 'seconds': 38}",5,19,DOWNS,2015,6,regular,False,True,42058,0.9973,2579,0.0027,3.178482,0


In [27]:
df

,offense,id,offense_conference,defense,defense_conference,home,away,offense_score,defense_score,drive_id,period,clock,yard_line,yards_to_goal,down,distance,yards_gained,play_type,ppa,id.1,game_id,scoring,start_period,start_yardline,start_yards_to_goal,start_time,end_period,end_yardline,end_yards_to_goal,end_time,elapsed,plays,yards,drive_result,season,week,season_type,neutral_site,conference_game,attendance,home_post_win_prob,away_id,away_post_win_prob,excitement_index,success_or_failure
0,Georgia,4.006040e+17,SEC,Auburn,SEC,Auburn ...,Georgia,0,7,4006039314,1,89,36,36,4,4,5,Pass Reception,2.130077,4006039314,400603931,False,1,66,66,"{'minutes': 4, 'seconds': 32}",2,2,2,"{'minutes': 12, 'seconds': 39}","{'minutes': 6, 'seconds': 53}",12,64,DOWNS,2015,11,regular,False,True,87451,0.6809,61,0.3191,3.706368,0
1,Vanderbilt,4.010120e+17,SEC,Tennessee State,None,Vanderbilt ...,Tennessee State,0,0,4010122992,1,696,68,32,4,2,-10,Pass Interception Return,-2.672218,4010122992,401012299,False,1,49,51,"{'minutes': 11, 'seconds': 36}",1,68,32,"{'minutes': 8, 'seconds': 17}","{'minutes': 3, 'seconds': 19}",6,19,INT,2018,5,regular,False,False,0,0.6333,2634,0.3667,6.214617,0
2,South Carolina,4.006040e+17,SEC,LSU,SEC,LSU ...,South Carolina,3,7,4006039033,1,315,56,56,4,2,0,Pass Incompletion,-0.589364,4006039033,400603903,False,1,75,75,"{'minutes': 5, 'seconds': 15}",1,56,56,"{'minutes': 2, 'seconds': 37}","{'minutes': 2, 'seconds': 38}",5,19,DOWNS,2015,6,regular,False,True,42058,0.9973,2579,0.0027,3.178482,0
3,Wake Forest,4.005480e+17,ACC,Army,FBS Independents,Wake Forest ...,Army,14,14,4005478328,2,562,65,35,4,11,3,Pass Reception,-1.287230,4005478328,400547832,False,2,38,62,"{'minutes': 9, 'seconds': 22}",2,68,32,"{'minutes': 6, 'seconds': 51}","{'minutes': 2, 'seconds': 31}",8,25,DOWNS,2014,4,regular,False,False,28123,0.5234,349,0.4766,5.788620,0
4,Kansas State,4.005480e+17,Big 12,Auburn,SEC,Kansas State ...,Auburn,0,3,4005478564,1,482,47,53,4,1,6,Penalty,0.000000,4005478564,400547856,False,1,25,75,"{'minutes': 11, 'seconds': 14}",1,80,20,"{'minutes': 5, 'seconds': 21}","{'minutes': 5, 'seconds': 53}",11,74,INT,2014,4,regular,False,False,53046,0.1775,2,0.8225,6.364702,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9358,Florida,4.005480e+17,SEC,Eastern Michigan,Mid-American,Florida ...,Eastern Michigan,0,0,4005481012,1,474,76,24,4,2,3,Rush,1.059424,4005481012,400548101,True,1,21,79,"{'minutes': 13, 'seconds': 15}",1,84,16,"{'minutes': 6, 'seconds': 44}","{'minutes': 6, 'seconds': 31}",16,63,FG,2014,2,regular,False,False,81049,1.0000,2199,0.0000,1.374711,1
9359,Virginia,4.005480e+17,ACC,Miami,ACC,Virginia ...,Miami,0,7,4005477937,2,900,92,8,4,2,3,Rush,1.093967,4005477937,400547793,True,1,33,67,"{'minutes': 3, 'seconds': 3}",2,97,3,"{'minutes': 11, 'seconds': 16}","{'minutes': 6, 'seconds': 47}",13,64,FG,2014,13,regular,False,True,39112,0.9870,2390,0.0130,3.689861,1
9360,Georgia,4.005480e+17,SEC,Auburn,SEC,Georgia ...,Auburn,14,7,4005483377,2,528,91,9,4,1,9,Rushing Touchdown,2.943088,4005483377,400548337,True,2,17,83,"{'minutes': 13, 'seconds': 35}",2,100,0,"{'minutes': 8, 'seconds': 48}","{'minutes': 4, 'seconds': 47}",11,83,TD,2014,12,regular,False,True,92746,0.9993,2,0.0007,3.007639,1
9361,Missouri,4.005480e+17,SEC,Arkansas,SEC,Missouri ...,Arkansas,0,7,4005483253,1,73,71,29,4,1,2,Rush,1.203179,4005483253,400548325,True,1,25,75,"{'minutes': 6, 'seconds': 48}",2,66,34,"{'minutes': 14, 'seconds': 23}","{'minutes': 7, 'seconds': 25}",17,41,FG,2014,14,regular,False,True,71168,0.5365,8,0.4635,5.696126,1


In [0]:
#set away or home as 0 or 1 categorical
home=[]
for num in range(len(df)):
  #print(df['offense'].iloc[num],len(df['offense'].iloc[num]),df['home'].iloc[num],len(df['home'].iloc[num]))
  hometeam=df['home'].iloc[num]
  if df['offense'].iloc[num]==hometeam [:len(df['offense'].iloc[num])]:
    home.append(1)
  else:
    home.append(0)
df['home']=home
df.drop('away',axis=1, inplace=True)

In [29]:
df

,offense,id,offense_conference,defense,defense_conference,home,offense_score,defense_score,drive_id,period,clock,yard_line,yards_to_goal,down,distance,yards_gained,play_type,ppa,id.1,game_id,scoring,start_period,start_yardline,start_yards_to_goal,start_time,end_period,end_yardline,end_yards_to_goal,end_time,elapsed,plays,yards,drive_result,season,week,season_type,neutral_site,conference_game,attendance,home_post_win_prob,away_id,away_post_win_prob,excitement_index,success_or_failure
0,Georgia,4.006040e+17,SEC,Auburn,SEC,0,0,7,4006039314,1,89,36,36,4,4,5,Pass Reception,2.130077,4006039314,400603931,False,1,66,66,"{'minutes': 4, 'seconds': 32}",2,2,2,"{'minutes': 12, 'seconds': 39}","{'minutes': 6, 'seconds': 53}",12,64,DOWNS,2015,11,regular,False,True,87451,0.6809,61,0.3191,3.706368,0
1,Vanderbilt,4.010120e+17,SEC,Tennessee State,None,1,0,0,4010122992,1,696,68,32,4,2,-10,Pass Interception Return,-2.672218,4010122992,401012299,False,1,49,51,"{'minutes': 11, 'seconds': 36}",1,68,32,"{'minutes': 8, 'seconds': 17}","{'minutes': 3, 'seconds': 19}",6,19,INT,2018,5,regular,False,False,0,0.6333,2634,0.3667,6.214617,0
2,South Carolina,4.006040e+17,SEC,LSU,SEC,0,3,7,4006039033,1,315,56,56,4,2,0,Pass Incompletion,-0.589364,4006039033,400603903,False,1,75,75,"{'minutes': 5, 'seconds': 15}",1,56,56,"{'minutes': 2, 'seconds': 37}","{'minutes': 2, 'seconds': 38}",5,19,DOWNS,2015,6,regular,False,True,42058,0.9973,2579,0.0027,3.178482,0
3,Wake Forest,4.005480e+17,ACC,Army,FBS Independents,1,14,14,4005478328,2,562,65,35,4,11,3,Pass Reception,-1.287230,4005478328,400547832,False,2,38,62,"{'minutes': 9, 'seconds': 22}",2,68,32,"{'minutes': 6, 'seconds': 51}","{'minutes': 2, 'seconds': 31}",8,25,DOWNS,2014,4,regular,False,False,28123,0.5234,349,0.4766,5.788620,0
4,Kansas State,4.005480e+17,Big 12,Auburn,SEC,1,0,3,4005478564,1,482,47,53,4,1,6,Penalty,0.000000,4005478564,400547856,False,1,25,75,"{'minutes': 11, 'seconds': 14}",1,80,20,"{'minutes': 5, 'seconds': 21}","{'minutes': 5, 'seconds': 53}",11,74,INT,2014,4,regular,False,False,53046,0.1775,2,0.8225,6.364702,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9358,Florida,4.005480e+17,SEC,Eastern Michigan,Mid-American,1,0,0,4005481012,1,474,76,24,4,2,3,Rush,1.059424,4005481012,400548101,True,1,21,79,"{'minutes': 13, 'seconds': 15}",1,84,16,"{'minutes': 6, 'seconds': 44}","{'minutes': 6, 'seconds': 31}",16,63,FG,2014,2,regular,False,False,81049,1.0000,2199,0.0000,1.374711,1
9359,Virginia,4.005480e+17,ACC,Miami,ACC,1,0,7,4005477937,2,900,92,8,4,2,3,Rush,1.093967,4005477937,400547793,True,1,33,67,"{'minutes': 3, 'seconds': 3}",2,97,3,"{'minutes': 11, 'seconds': 16}","{'minutes': 6, 'seconds': 47}",13,64,FG,2014,13,regular,False,True,39112,0.9870,2390,0.0130,3.689861,1
9360,Georgia,4.005480e+17,SEC,Auburn,SEC,1,14,7,4005483377,2,528,91,9,4,1,9,Rushing Touchdown,2.943088,4005483377,400548337,True,2,17,83,"{'minutes': 13, 'seconds': 35}",2,100,0,"{'minutes': 8, 'seconds': 48}","{'minutes': 4, 'seconds': 47}",11,83,TD,2014,12,regular,False,True,92746,0.9993,2,0.0007,3.007639,1
9361,Missouri,4.005480e+17,SEC,Arkansas,SEC,1,0,7,4005483253,1,73,71,29,4,1,2,Rush,1.203179,4005483253,400548325,True,1,25,75,"{'minutes': 6, 'seconds': 48}",2,66,34,"{'minutes': 14, 'seconds': 23}","{'minutes': 7, 'seconds': 25}",17,41,FG,2014,14,regular,False,True,71168,0.5365,8,0.4635,5.696126,1


In [35]:
df_drives=pd.read_csv('https://jasonstewart.s3.amazonaws.com/drives.csv')
df_drives

for col in df_drives:
  name='next_drive_'+col
  df_drives.rename(columns={col : name}, inplace=True)
df_drives

for col in df_drives:
  df[col]=""

for num in range(len(df)):
  drive_id=df['id.1'].iloc[num]+1
  #print(drive_id,df['id.1'].iloc[num])
  next_drive=df_drives[df_drives['next_drive_id']==drive_id]
  #print(next_drive['next_drive_id'].iloc[0],df['id.1'].iloc[num])
  if not next_drive.empty:
    for col in next_drive:
      #print(next_drive['next_drive_id'].iloc[0],df['id.1'].iloc[num])
      df[col].iloc[num]=next_drive[col].iloc[0]
print(df.columns)
print(df_drives.columns)

#df_drives.drop(['offense','offense_conference','defense','defense_conference' ] ,axis=1)
df['drive_id']=df['id.1']
df.drop(['id.1'],axis=1,inplace=True)
df['drive_id']


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Index(['offense', 'id', 'offense_conference', 'defense', 'defense_conference',
       'home', 'offense_score', 'defense_score', 'drive_id', 'period', 'clock',
       'yard_line', 'yards_to_goal', 'down', 'distance', 'yards_gained',
       'play_type', 'ppa', 'id.1', 'game_id', 'scoring', 'start_period',
       'start_yardline', 'start_yards_to_goal', 'start_time', 'end_period',
       'end_yardline', 'end_yards_to_goal', 'end_time', 'elapsed', 'plays',
       'yards', 'drive_result', 'season', 'week', 'season_type',
       'neutral_site', 'conference_game', 'attendance', 'home_post_win_prob',
       'away_id', 'away_post_win_prob', 'excitement_index',
       'success_or_failure', 'next_drive_id', 'next_drive_game_id',
       'next_drive_offense', 'next_drive_offense_conference',
       'next_drive_defense', 'next_drive_defense_conference',
       'next_drive_scoring', 'next_drive_start_period',
       'next_drive_start_yardline', 'next_drive_start_yards_to_goal',
       'next_drive_sta

,offense,id,offense_conference,defense,defense_conference,home,offense_score,defense_score,drive_id,period,clock,yard_line,yards_to_goal,down,distance,yards_gained,play_type,ppa,game_id,scoring,start_period,start_yardline,start_yards_to_goal,start_time,end_period,end_yardline,end_yards_to_goal,end_time,elapsed,plays,yards,drive_result,season,week,season_type,neutral_site,conference_game,attendance,home_post_win_prob,away_id,away_post_win_prob,excitement_index,success_or_failure,next_drive_id,next_drive_game_id,next_drive_offense,next_drive_offense_conference,next_drive_defense,next_drive_defense_conference,next_drive_scoring,next_drive_start_period,next_drive_start_yardline,next_drive_start_yards_to_goal,next_drive_start_time,next_drive_yard_line,next_drive_yards_to_goal,next_drive_end_period,next_drive_end_yardline,next_drive_end_yards_to_goal,next_drive_end_time,next_drive_elapsed,next_drive_plays,next_drive_yards,next_drive_drive_result
0,Georgia,4.006040e+17,SEC,Auburn,SEC,0,0,7,4006039314,1,89,36,36,4,4,5,Pass Reception,2.130077,400603931,False,1,66,66,"{'minutes': 4, 'seconds': 32}",2,2,2,"{'minutes': 12, 'seconds': 39}","{'minutes': 6, 'seconds': 53}",12,64,DOWNS,2015,11,regular,False,True,87451,0.6809,61,0.3191,3.706368,0,4006039315,400603931,Auburn,SEC,Georgia,SEC,False,2,2,98,"{'minutes': 12, 'seconds': 39}",NaN,NaN,2,50,50,"{'minutes': 10, 'seconds': 26}","{'minutes': 2, 'seconds': 13}",5,48,INT
1,Vanderbilt,4.010120e+17,SEC,Tennessee State,None,1,0,0,4010122992,1,696,68,32,4,2,-10,Pass Interception Return,-2.672218,401012299,False,1,49,51,"{'minutes': 11, 'seconds': 36}",1,68,32,"{'minutes': 8, 'seconds': 17}","{'minutes': 3, 'seconds': 19}",6,19,INT,2018,5,regular,False,False,0,0.6333,2634,0.3667,6.214617,0,4010122993,401012299,Tennessee State,None,Vanderbilt,SEC,False,1,59,59,"{'minutes': 8, 'seconds': 17}",NaN,NaN,1,29,29,"{'minutes': 5, 'seconds': 8}","{'minutes': 3, 'seconds': 9}",8,30,MISSED FG
2,South Carolina,4.006040e+17,SEC,LSU,SEC,0,3,7,4006039033,1,315,56,56,4,2,0,Pass Incompletion,-0.589364,400603903,False,1,75,75,"{'minutes': 5, 'seconds': 15}",1,56,56,"{'minutes': 2, 'seconds': 37}","{'minutes': 2, 'seconds': 38}",5,19,DOWNS,2015,6,regular,False,True,42058,0.9973,2579,0.0027,3.178482,0,4006039034,400603903,LSU,SEC,South Carolina,SEC,False,1,57,43,"{'minutes': 2, 'seconds': 37}",NaN,NaN,2,64,36,"{'minutes': 14, 'seconds': 40}","{'minutes': 2, 'seconds': 57}",7,7,PUNT
3,Wake Forest,4.005480e+17,ACC,Army,FBS Independents,1,14,14,4005478328,2,562,65,35,4,11,3,Pass Reception,-1.287230,400547832,False,2,38,62,"{'minutes': 9, 'seconds': 22}",2,68,32,"{'minutes': 6, 'seconds': 51}","{'minutes': 2, 'seconds': 31}",8,25,DOWNS,2014,4,regular,False,False,28123,0.5234,349,0.4766,5.788620,0,4005478329,400547832,Army,FBS Independents,Wake Forest,ACC,True,2,68,68,"{'minutes': 6, 'seconds': 51}",NaN,NaN,2,0,0,"{'minutes': 2, 'seconds': 23}","{'minutes': 4, 'seconds': 28}",9,68,TD
4,Kansas State,4.005480e+17,Big 12,Auburn,SEC,1,0,3,4005478564,1,482,47,53,4,1,6,Penalty,0.000000,400547856,False,1,25,75,"{'minutes': 11, 'seconds': 14}",1,80,20,"{'minutes': 5, 'seconds': 21}","{'minutes': 5, 'seconds': 53}",11,74,INT,2014,4,regular,False,False,53046,0.1775,2,0.8225,6.364702,0,4005478565,400547856,Auburn,SEC,Kansas State,Big 12,False,1,80,80,"{'minutes': 5, 'seconds': 21}",NaN,NaN,1,57,57,"{'minutes': 3, 'seconds': 41}","{'minutes': 1, 'seconds': 40}",4,16,PUNT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9358,Florida,4.005480e+17,SEC,Eastern Michigan,Mid-American,1,0,0,4005481012,1,474,76,24,4,2,3,Rush,1.059424,400548101,True,1,21,79,"{'minutes': 13, 'seconds': 15}",1,84,16,"{'minutes': 6, 'seconds': 44}","{'minutes': 6, 'seconds': 31}",16,63,FG,2014,2,regular,False,False,81049,1.0000,2199,0.0000,1.374711,1,400

In [37]:
print(df['drive_id'],df['next_drive_id'])

0       4006039314
1       4010122992
2       4006039033
3       4005478328
4       4005478564
           ...    
9358    4005481012
9359    4005477937
9360    4005483377
9361    4005483253
9362    4006038287
Name: drive_id, Length: 9363, dtype: int64 0       4006039315
1       4010122993
2       4006039034
3       4005478329
4       4005478565
           ...    
9358    4005481013
9359    4005477938
9360    4005483378
9361    4005483254
9362    4006038288
Name: next_drive_id, Length: 9363, dtype: object


In [0]:
df.to_csv('college-football-data-p5-4th-down.csv')